# Manipulate the Model Data

This example shows how to manipulate the model.

In [1]:
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2023-11-07 22:31:12
ams:0.7.3.post29.dev0+g23b6565


In [3]:
ams.config_logger(stream_level=20)

## Run Simulations

### Load Case

In [4]:
sp = ams.load(ams.get_case('ieee39/ieee39_uced.xlsx'),
              default_config=True,
              setup=True)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee39/ieee39_uced.xlsx"...
Input file parsed in 0.1714 seconds.
System set up in 0.0043 seconds.


The system load are defined in model `PQ`.

In [5]:
sp.PQ.as_df()

,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner
uid,,,,,,,,,,
0,PQ_1,1.0,PQ_1,3,345.0,6.000,2.500,1.2,0.8,1
1,PQ_2,1.0,PQ_2,4,345.0,4.500,1.840,1.2,0.8,1
2,PQ_3,1.0,PQ_3,7,345.0,2.338,0.840,1.2,0.8,1
3,PQ_4,1.0,PQ_4,8,345.0,5.220,1.766,1.2,0.8,1
4,PQ_5,1.0,PQ_5,12,138.0,1.200,0.300,1.2,0.8,1
5,PQ_6,1.0,PQ_6,15,345.0,3.200,1.530,1.2,0.8,1
6,PQ_7,1.0,PQ_7,16,345.0,3.290,0.323,1.2,0.8,1
7,PQ_8,1.0,PQ_8,18,345.0,1.580,0.300,1.2,0.8,1
8,PQ_9,1.0,PQ_9,20,138.0,6.800,1.030,1.2,0.8,1


For simplicity, PQ is reorganized as nodal load `pl` in an routine.

In [6]:
sp.RTED.pl.v

array([0.   , 0.   , 6.   , 4.5  , 0.   , 0.   , 2.338, 5.22 , 0.   ,
       0.   , 0.   , 1.2  , 0.   , 0.   , 3.2  , 3.29 , 0.   , 1.58 ,
       0.   , 6.8  , 2.74 , 0.   , 2.475, 3.086, 2.24 , 1.39 , 2.81 ,
       2.06 , 2.835, 0.   , 0.8  , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 4.   ])

RTED can be solved and one can inspect the results as discussed in
previous example.

In [7]:
sp.RTED.run()

Routine <RTED> initialized in 0.0083 seconds.
RTED solved as optimal in 0.0138 seconds, converged after 50 iterations using solver OSQP.


True

In [8]:
sp.RTED.pg.v

array([6.01822025, 5.99427957, 5.97043409, 5.08      , 5.98234496,
       5.8       , 5.64      , 6.00623798, 6.03022646, 6.04225669])

The load values can be manipulated in the model `PQ`.

In [9]:
sp.PQ.set(src='p0', attr='v', idx=['PQ_1', 'PQ_3'], value=[6.5, 3])

True

The routine need to be re-initialized to make the changes effective.

In [10]:
sp.RTED.init()

Routine <RTED> initialized in 0.0064 seconds.


True

We can see the `pl` is changed as expected.

In [11]:
sp.RTED.pl.v

array([0.   , 0.   , 6.5  , 4.5  , 0.   , 0.   , 3.   , 5.22 , 0.   ,
       0.   , 0.   , 1.2  , 0.   , 0.   , 3.2  , 3.29 , 0.   , 1.58 ,
       0.   , 6.8  , 2.74 , 0.   , 2.475, 3.086, 2.24 , 1.39 , 2.81 ,
       2.06 , 2.835, 0.   , 0.8  , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 4.   ])

After manipulation, the routined can be solved again.

In [12]:
sp.RTED.run()

RTED solved as optimal in 0.0140 seconds, converged after 50 iterations using solver OSQP.


True

In [13]:
sp.RTED.pg.v

array([6.18438538, 6.16011369, 6.13593852, 5.07999972, 6.14801408,
       5.79999973, 5.63999972, 6.17223744, 6.19655759, 6.20875415])